In [1]:
import open3d as o3d
import numpy as np
from IPython.display import display, HTML

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
def load_point_cloud(file_path):
    """Load a point cloud from a file."""
    cloud = o3d.io.read_point_cloud(file_path)
    return cloud

def align_point_clouds(source_cloud, target_cloud):
    """Align two point clouds using ICP."""
    icp_transformation = o3d.pipelines.registration.registration_icp(
        source_cloud, target_cloud, max_correspondence_distance=25,
        estimation_method=o3d.pipelines.registration.TransformationEstimationPointToPoint(),
        criteria=o3d.pipelines.registration.ICPConvergenceCriteria(max_iteration=1000)
    )

    aligned_source_cloud = source_cloud.transform(icp_transformation.transformation)
    return aligned_source_cloud

def save_point_cloud(file_path, point_cloud):
    """Save a point cloud to a file."""
    o3d.io.write_point_cloud(file_path, point_cloud)

def visualize_pointcloud(pointcloud):
    vis = o3d.visualization.Visualizer()
    vis.create_window()
    vis.add_geometry(pointcloud)

    # Capture the generated image and display it in the notebook
    image = vis.capture_screen_float_buffer(True)
    display(o3d.visualization.draw_geometries_with_animation_callback([pointcloud], vis, interval=50))
    
    # Clean up resources after visualization
    vis.destroy_window()

def downsample_pointcloud(pointcloud, voxel_size):
    # Voxel downsample the point cloud
    downsampled_pc = pointcloud.voxel_down_sample(voxel_size)
    return downsampled_pc
    

In [14]:
source_path = "../output/bim.ply"
#target_path = "../output/pcfull.ply" #Pointcloud from depth using groundtruth poses
target_path = "../output/pcfull_off.ply" #Pointcloud from depth using offset groundtruth poses


source_cloud = load_point_cloud(source_path)
target_cloud = load_point_cloud(target_path)

In [16]:
o3d.visualization.draw_geometries([source_cloud, target_cloud ])

In [17]:
source_cloud = np.asarray(source_cloud.points).astype(np.float32)
target_cloud = np.asarray(target_cloud.points).astype(np.float32)

In [18]:
source_center = np.mean(source_cloud, axis=0)
print('Source Center: ', source_center)
target_center = np.mean(target_cloud, axis=0)
print('Target Center: ',target_center)
diff = target_center - source_center
print('Offset: ', diff)
source_cloud_recentered = source_cloud + diff

Source Center:  [163.78734   99.141884  11.349497]
Target Center:  [25.860271  12.604036   5.8141937]
Offset:  [-137.92706   -86.53785    -5.535303]


In [19]:
source_cloud_recentered_o3 = o3d.geometry.PointCloud()
source_cloud_recentered_o3.points = o3d.utility.Vector3dVector(source_cloud_recentered)

target_cloud_o3 = o3d.geometry.PointCloud()
target_cloud_o3.points = o3d.utility.Vector3dVector(target_cloud)

In [20]:
o3d.visualization.draw_geometries([target_cloud_o3, source_cloud_recentered_o3])

In [21]:
aligned_source_cloud = align_point_clouds(source_cloud_recentered_o3, target_cloud_o3)

In [22]:
o3d.visualization.draw_geometries([aligned_source_cloud,  target_cloud_o3])

In [24]:
aligned_ply_path = "../output/aligned_off.ply"
save_point_cloud(aligned_ply_path, aligned_source_cloud)